# Feature Selection (2004)
This notebook follows the same process detailed in Feature Selection (2012), but with 2008 as the target year.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.metrics import f1_score, confusion_matrix, mutual_info_score, roc_auc_score
from helpers.machine_learning import Normalizer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, chi2, RFECV, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from scipy.stats import pearsonr

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = 50
pd.options.display.max_rows = 50

In [2]:
df_orig = pd.read_csv('../data/anes_cdf_converted.csv')
df_orig.head()

,Unnamed: 0,year,age,congressional_district,state,gender,weight,final_vote,VCF0108,VCF0113,VCF0127,VCF0143,VCF0146,VCF0311,VCF0346,VCF0347,VCF0348,VCF0349,VCF0358,VCF0359,VCF0360,VCF0361,VCF0370,VCF0371,VCF0372,...,VCF0904_oh2,VCF0904_oh3,VCF1004_oh0,VCF1004_oh1,VCF1004_oh2,VCF1004_oh3,VCF1004_oh4,VCF9030_oh0,VCF9030_oh1,VCF9030_oh2,VCF9030_oh3,VCF9030_oh4,VCF9030_oh5,VCF9131_oh0,VCF9131_oh1,VCF9131_oh2,VCF9131_oh3,VCF9132_oh0,VCF9132_oh1,VCF9132_oh2,VCF9132_oh3,VCF9133_oh0,VCF9133_oh1,VCF9133_oh2,VCF9133_oh3
0,0,2000,49.0,MN01,MN,0,1.2886,2,0.0,0,0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1,2000,35.0,MI01,MI,1,0.8959,0,0.0,0,0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,2,2000,57.0,IL11,IL,1,1.0454,1,0.0,0,0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,2000,63.0,ME02,ME,0,0.6005,1,0.0,0,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,4,2000,40.0,MA01,MA,1,1.9270,2,0.0,0,0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [3]:
def cv_test(X_train, y_train, preprocessing, classifiers, clf_names, scoring = 'f1', cv = 5):
    scores = []
    if preprocessing != None:
        X_cv = preprocessing.fit_transform(X_train)
    else:
        X_cv = X_train
    for model, name in zip(classifiers, clf_names):
        cv_score = cross_val_score(X = X_cv, y = y_train, estimator = model, cv = cv, scoring = scoring)
        scores.append([cv_score.mean(), cv_score.ptp(), cv_score.std()])
    scores = pd.DataFrame(scores, columns = ['mean','range','std'])
    scores.index = clf_names
    return scores.sort_values(by = 'mean', ascending = False)

In [4]:
df_orig = df_orig.drop(['Unnamed: 0', 'congressional_district','state','final_vote'], axis = 1)

df_orig = df_orig.iloc[:, ~df_orig.columns.str.contains('VCF0734')]
df_orig = df_orig.iloc[:, ~df_orig.columns.str.contains('VCF0736')]
df_orig = df_orig.iloc[:, ~df_orig.columns.str.contains('VCF1011')]
df_orig = df_orig.iloc[:, ~df_orig.columns.str.contains('VCF0704')]
df_orig = df_orig.iloc[:, ~df_orig.columns.str.contains('VCF0710')]
df_orig = df_orig.iloc[:, ~df_orig.columns.str.contains('VCF0709')]
df_orig = df_orig.iloc[:, ~df_orig.columns.str.contains('VCF0703')]
df_orig = df_orig.iloc[:, ~df_orig.columns.str.contains('VCF0707')]
df_orig = df_orig.iloc[:, ~df_orig.columns.str.contains('VCF0708')]
df_orig = df_orig.iloc[:, ~df_orig.columns.str.contains('VCF1011')]

# Label non-voters as positive cases and voters as negative cases
df_orig.VCF0702 = df_orig.VCF0702.apply(lambda x: 0 if x==1 else 1)

Separate training (pre-2004) and test (2004) data so that we can test the generalizability of a model trained on data from earlier years to that of a target year.

In [5]:
df = df_orig[:]
df_train = df[df.year < 2004]

todrop = ['VCF0702','year']
X_train_orig = df_train.drop(todrop, axis = 1)
y_train = df_train.VCF0702

columns = X_train_orig.columns

Test various classifiers on training data

In [6]:
imp = Imputer(missing_values = 'NaN', strategy = 'median')

preprocessing = Pipeline([('imp',imp),('scale', Normalizer())])

classifiers = [LogisticRegression(), RandomForestClassifier(), AdaBoostClassifier(),
               BernoulliNB(), GaussianNB(), SVC()]
clf_names = ['Logistic Regression','Random Forest', 'AdaBoost',
             'Bernoulli Naive Bayes','Gaussian Naive Bayes', 'SVM']

scores = cv_test(X_train_orig, y_train, preprocessing, classifiers, clf_names)
scores

,mean,range,std
Bernoulli Naive Bayes,0.632860,0.103270,0.036795
AdaBoost,0.624909,0.252535,0.085224
Logistic Regression,0.548718,0.081081,0.028947
SVM,0.532934,0.211870,0.075687
Gaussian Naive Bayes,0.479487,0.116795,0.043043
Random Forest,0.426254,0.522648,0.191106


Test effect of PCA on training data with various classifiers

In [8]:
preprocessing = Pipeline([('imp', imp),('scale', Normalizer()),('pca', PCA(n_components = 200))])
scores = cv_test(X_train_orig, y_train, preprocessing, classifiers, clf_names)
scores

,mean,range,std
Gaussian Naive Bayes,0.554722,0.191851,0.069382
Logistic Regression,0.551286,0.111688,0.042289
SVM,0.550438,0.233696,0.082542
AdaBoost,0.452281,0.219852,0.080475
Bernoulli Naive Bayes,0.305333,0.280420,0.106310
Random Forest,0.215494,0.157265,0.052157


# Adding new features

### Adding thermometer intensity

In [9]:
def add_thermometer_intensity(df):
    
    def thermometer_to_intensity(x):
        if x == np.nan:
            return np.nan
        else:
            return abs(x - 5) ** 2
    
    columns_to_convert = (df.max() >= 10) & (df.min() == 0)
    columns_to_convert[[df.columns.get_loc('VCF0114_r1'),df.columns.get_loc('VCF1015')]] = False
    thermometer_df = df.loc[:,columns_to_convert]
    thermometer_df = thermometer_df.applymap(thermometer_to_intensity)
    thermometer_df.columns = thermometer_df.columns + '_int'
    thermometer_df['int_sum_therm'] = thermometer_df.sum(axis = 1) ** 1.2
    return pd.concat([df, thermometer_df], axis = 1)

In [10]:
X_train = add_thermometer_intensity(X_train_orig)
preprocessing = Pipeline([('imp', Imputer(missing_values = 'NaN', strategy = 'median')),('scale', Normalizer())])
scores = cv_test(X_train, y_train, preprocessing = preprocessing, classifiers = classifiers, clf_names = clf_names)
scores

,mean,range,std
Bernoulli Naive Bayes,0.637497,0.046130,0.015059
AdaBoost,0.618026,0.091855,0.032005
Logistic Regression,0.579163,0.119403,0.041224
SVM,0.504213,0.442857,0.161609
Gaussian Naive Bayes,0.488067,0.114899,0.048315
Random Forest,0.472080,0.138462,0.049760


### Adding ordinal intensity

In [11]:
def add_ordinal_intensity(df):
    columns_to_convert = ['VCF0803','VCF0806','VCF0830','VCF0851','VCF9014','VCF9015','VCF9039','VCF9042','VCF0301',
                     'VCF0303','VCF0502','VCF0604','VCF0605','VCF0880a','VCF9009','VCF9045']
    intensity_df = df.loc[:,columns_to_convert]
    intensity_df = abs(intensity_df - (intensity_df.max() + intensity_df.min()) / 2)
    intensity_df.columns = intensity_df.columns + '_int'
    intensity_df['int_sum_ord'] = intensity_df.sum(axis = 1) ** 2
    return pd.concat([df, intensity_df], axis = 1)

In [12]:
X_train = add_ordinal_intensity(X_train_orig)
scores = cv_test(X_train, y_train, preprocessing, classifiers, clf_names)
scores

,mean,range,std
AdaBoost,0.623276,0.298930,0.105569
Bernoulli Naive Bayes,0.621164,0.079174,0.027345
Logistic Regression,0.555819,0.134460,0.049927
SVM,0.502553,0.215873,0.076366
Random Forest,0.488653,0.269744,0.088908
Gaussian Naive Bayes,0.482893,0.121622,0.044669


### Summing "don't know" responses

In [13]:
def add_dk_sum(df):
    df = df[:]
    dk_column_index = df.columns.str.contains('dk')
    dk_df = df.loc[:,dk_column_index]
    df['dk_sum'] = dk_df.sum(axis = 1) ** 2
    return df

In [14]:
X_train = add_dk_sum(X_train_orig)
scores = cv_test(X_train, y_train, preprocessing, classifiers, clf_names)
scores

,mean,range,std
Bernoulli Naive Bayes,0.622441,0.131455,0.046716
AdaBoost,0.606321,0.193298,0.069687
Logistic Regression,0.549532,0.074047,0.029227
SVM,0.509203,0.306818,0.103554
Gaussian Naive Bayes,0.478764,0.121622,0.044760
Random Forest,0.464408,0.203228,0.068475


# Removing features
### First one-hot features

In [15]:
def drop_first_onehot(df):
    return df.loc[:, ~df.columns.str.contains('oh0')]

### Correlated features

In [16]:
def break_correlation(in_X, y, threshhold = 0.85, scoring = mutual_info_score):
       
    X = pd.DataFrame(Imputer(missing_values='NaN', strategy = 'median').fit_transform(in_X), columns = in_X.columns)
    
    corr_mask = abs(X.corr()) > threshhold
    corr_list = corr_mask.sum()
    corr_list = corr_list[corr_list > 1]
    corr_dict = dict(corr_list)
    
    print('Correlated features:', len(corr_dict))
    
    cluster_list = []
    while len(corr_list) > 0:
        cluster = dict()
        key = corr_list.index[0]
        correlations = list(corr_mask[key][corr_mask[key]].index)

        i = 1
        while i < len(correlations):
            key = correlations[i]
            temp_correlations = list(corr_mask[key][corr_mask[key]].index)
            difference = list(set(temp_correlations) - set(correlations))
            if len(difference) != 0:
                correlations = correlations + difference
            i = i + 1
            
        for i in range(0, len(correlations)):
            cluster[correlations[i]] = corr_dict[correlations[i]]
            corr_dict.pop(correlations[i])
            corr_list.pop(correlations[i])
            
        cluster_list.append(cluster)
        
    removed = []
        
    for cluster in cluster_list:
        scores = dict()
        for feature in cluster:
            scores[feature] = scoring(X[feature], y)

        while(len(cluster) > 0):
            min_key = min(scores, key = scores.get)
            removed.append(min_key)
            temp_correlations = list(corr_mask[min_key][corr_mask[min_key]].index)
            temp_correlations = list(set(temp_correlations).intersection(cluster))
            temp_correlations.remove(min_key)
            for each in temp_correlations:
                if cluster[each] > 2:
                    cluster[each] = cluster[each] - 1
                else:
                    cluster.pop(each)
                    scores.pop(each)
                                
            cluster.pop(min_key)
            scores.pop(min_key)
        
    print('Removed {} features:\n'.format(len(removed)),removed)        
    return in_X.drop(removed, axis = 1)

In [17]:
columns_to_drop = X_train_orig.describe().loc[:,X_train_orig.describe().loc['count',:] == 0].columns
X_train = X_train_orig.drop(columns_to_drop, axis = 1)
X_train = drop_first_onehot(X_train)
X_train = break_correlation(X_train, y_train)
scores = cv_test(X_train, y_train, preprocessing, classifiers, clf_names)
scores

Correlated features: 50
Removed 26 features:
 ['VCF0107_oh5', 'VCF0105a_oh5', 'VCF0112_oh2', 'VCF0127', 'VCF0450', 'VCF0904_oh1', 'VCF9030a', 'VCF0211_dk', 'VCF0429_dk', 'VCF0424', 'VCF0624', 'VCF0114_r1', 'VCF0504_dk', 'VCF0514_dk', 'VCF0541_dk', 'VCF0550_dk', 'VCF0804_dk', 'VCF0803_dk', 'VCF0804', 'VCF0110', 'VCF0303', 'VCF0870', 'VCF0714_oh2', 'VCF9131_oh2', 'VCF9132_oh1', 'VCF9133_oh2']


,mean,range,std
Bernoulli Naive Bayes,0.634529,0.040052,0.016370
AdaBoost,0.634490,0.201961,0.063956
Logistic Regression,0.559425,0.074047,0.027321
SVM,0.530918,0.269444,0.092739
Gaussian Naive Bayes,0.522423,0.153714,0.061080
Random Forest,0.434258,0.319540,0.113391


# Recursive feature elimination

In [18]:
def feature_elimination(X, y, preprocessing, estimator = LogisticRegression()):
    X = preprocessing.fit_transform(X)
    rfecv = RFECV(estimator = estimator, step = 5, cv = StratifiedKFold(3), scoring = 'f1')
    return rfecv.fit_transform(X, y), rfecv

In [19]:
X_train = X_train_orig
columns = X_train.columns
X_train, rfecv = feature_elimination(X_train, y_train, preprocessing, LogisticRegression())
print('Number of features selected:', rfecv.n_features_)
scores = cv_test(X_train, y_train, None, classifiers, clf_names)
scores

Number of features selected: 17


,mean,range,std
Bernoulli Naive Bayes,0.688104,0.103541,0.040688
AdaBoost,0.665896,0.060732,0.022746
Gaussian Naive Bayes,0.646794,0.116143,0.041521
Logistic Regression,0.642216,0.130046,0.046931
Random Forest,0.584487,0.257996,0.109928
SVM,0.557008,0.154545,0.066779


### Most important features

In [20]:
selected_features = columns[rfecv.ranking_ == 1]
X_rfe = break_correlation(pd.DataFrame(X_train, columns = selected_features), y_train)

Correlated features: 0
Removed 0 features:
 []


In [21]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [22]:
feature_rankings = pd.concat([pd.DataFrame(lr.coef_.T), pd.DataFrame(selected_features)], axis = 1)
feature_rankings.columns = ['coefficient','feature']
feature_rankings.coefficient = feature_rankings.coefficient.apply(lambda x: abs(x))
feature_rankings['sum'] = feature_rankings.feature.apply(lambda x: sum(df_train.loc[:,x].dropna()))
feature_rankings.sort_values(by = 'coefficient', ascending = False).head(50)

,coefficient,feature,sum
14,1.899820,VCF0302_oh2,194.0
1,1.342136,VCF0311,571.0
3,1.164380,VCF9031,82.0
2,1.138243,VCF9029,0.0
5,1.111918,VCF0223_dk,43.0
4,1.082310,VCF0211,36045.0
16,0.985632,VCF0904_oh2,548.0
8,0.968151,VCF9005,44570.0
15,0.934805,VCF0302_oh5,247.0
11,0.917805,VCF0853,1178.0


### Classifying with VCF0713_oh4 as only feature

In [23]:
df = df_orig[:]
X_train = df[df.year < 2004]
score_cv = cross_val_score(lr, X = X_train.VCF0713_oh4.reshape(-1,1), y = y_train, scoring = 'f1', cv = 10)
print('f1-score:', score_cv.mean())

f1-score: 0.513703327324


### Classifying with everything else

In [24]:
df = df_orig[:]
X_train = df[df.year < 2004]
X_train = X_train.drop(['VCF0713_oh4','VCF0713_oh3','VCF0713_oh2','VCF0713_oh1','VCF0702'], axis = 1)
X_exp = preprocessing.fit_transform(X_train)
score_cv = cross_val_score(lr, X = X_exp, y = y_train, scoring = 'f1', cv = 10)
score_cv.mean()

0.53745708356002475

# Testing Combinations

### Optimizing feature sets for voting classifiers

In [50]:
max_f1 = [0,0,0,0]
best_config = [0,0,0,0]
X_train_features = [0,0,0,0]

classifiers = [LogisticRegression(), AdaBoostClassifier(), BernoulliNB(), SVC()]
clf_names = ['Logistic Regression', 'AdaBoost', 'Bernoulli Naive Bayes', 'SVM']

for i in range(0, 32):
    columns_to_drop = X_train_orig.describe().loc[:,X_train_orig.describe().loc['count',:] == 0].columns
    X_train = X_train_orig.drop(columns_to_drop, axis = 1)
    operations = []
    flags = '{:05d}'.format(int(bin(i)[2:]))
    print('flags:', flags)
    if int(flags[0]):
        X_train = add_thermometer_intensity(X_train)
        operations.append('add thermometer intensity')
    if int(flags[1]):
        X_train = add_ordinal_intensity(X_train)
        operations.append('add ordinal intensity')
    if int(flags[2]):
        X_train = add_dk_sum(X_train)
        operations.append('add dk sum')
    if int(flags[3]):        
        X_train = drop_first_onehot(X_train)
        operations.append('drop first one-hot')
    if int(flags[4]):
        columns_to_drop = X_train.describe().loc[:,X_train.describe().loc['count',:] == 0].columns
        X_train = X_train.drop(columns_to_drop, axis = 1)
        X_train = break_correlation(X_train, y_train)
        operations.append('break correlation')
    columns = X_train.columns
    X_train, rfecv = feature_elimination(X_train, y_train, preprocessing, LogisticRegression())
    print('Operations:', operations)
    print('Number of features selected:', rfecv.n_features_)
    scores = cv_test(X_train, y_train, None, classifiers, clf_names, cv = 5)
    print(scores.sort_values(by = 'mean', ascending = False))
    
    selected_features = columns[rfecv.ranking_ == 1]
    for i in range(0,4):
        if max_f1[i] < scores['mean'][i]:
            max_f1[i] = scores['mean'][i]
            best_config[i] = operations
            X_train_features[i] = selected_features
    print('')

for f1, config, name in zip(max_f1, best_config, clf_names):
    print('best f1 for {} is {} with combination: {}'.format(name, f1, config))

flags: 00000
Operations: []
Number of features selected: 17
                           mean     range       std
Bernoulli Naive Bayes  0.688104  0.103541  0.040688
AdaBoost               0.665896  0.060732  0.022746
Logistic Regression    0.642216  0.130046  0.046931
SVM                    0.557008  0.154545  0.066779

flags: 00001
Correlated features: 50
Removed 26 features:
 ['VCF0107_oh5', 'VCF0105a_oh5', 'VCF0112_oh2', 'VCF0127', 'VCF0450', 'VCF0904_oh1', 'VCF9030a', 'VCF0211_dk', 'VCF0429_dk', 'VCF0424', 'VCF0624', 'VCF0114_r1', 'VCF0504_dk', 'VCF0514_dk', 'VCF0541_dk', 'VCF0550_dk', 'VCF0804_dk', 'VCF0803_dk', 'VCF0804', 'VCF0110', 'VCF0303', 'VCF0870', 'VCF0714_oh2', 'VCF9131_oh2', 'VCF9132_oh1', 'VCF9133_oh2']
Operations: ['break correlation']
Number of features selected: 51
                           mean     range       std
Bernoulli Naive Bayes  0.670280  0.123509  0.046842
Logistic Regression    0.654434  0.146269  0.049701
AdaBoost               0.649852  0.101695  0.03889

In [51]:
X_train_all = add_dk_sum(add_thermometer_intensity(add_ordinal_intensity(X_train_orig)))
X_train_config = [0,0,0,0]
for i in range(0, len(X_train_config)):
    X_train_config[i] = X_train_all.loc[:,X_train_features[i]]

print(cv_test(X_train_config[0], y_train, preprocessing, [LogisticRegression()], ['Logistic Regression'], cv = 10),'\n')
print(cv_test(X_train_config[1], y_train, preprocessing, [AdaBoostClassifier()], ['AdaBoost'], cv = 10),'\n')
print(cv_test(X_train_config[2], y_train, preprocessing, [BernoulliNB()], ['Bernoulli Naive Bayes'], cv = 10),'\n')
print(cv_test(X_train_config[3], y_train, preprocessing, [SVC()], ['SVM'], cv = 10),'\n')

                        mean    range       std
Logistic Regression  0.64055  0.54902  0.131831 

              mean     range       std
AdaBoost  0.687163  0.310924  0.102909 

                           mean     range       std
Bernoulli Naive Bayes  0.699026  0.292292  0.087139 

         mean     range       std
SVM  0.592865  0.512987  0.141275 



In [52]:
for i in range(0,len(X_train_config)):
    print(X_train_config[i].shape)

(718, 16)
(718, 20)
(718, 20)
(718, 32)


In [53]:
X_train_config[0].to_csv('../data/anes_cdf_training_2004_lr.csv')
X_train_config[1].to_csv('../data/anes_cdf_training_2004_ada.csv')
X_train_config[2].to_csv('../data/anes_cdf_training_2004_bnb.csv')
X_train_config[3].to_csv('../data/anes_cdf_training_2004_svm.csv')